У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [185]:
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE, SMOTENC
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.multiclass import OneVsRestClassifier
from imblearn.pipeline import Pipeline as ImbPipeline

from modules.modules_2.topic_2_1.utils import analyze_dataframe

In [186]:
raw_df = pd.read_csv('customer_segmentation_train.csv')

In [187]:
raw_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [188]:
analyze_dataframe(raw_df.drop(columns=['ID']))

Кількість рядків: 8068
Кількість колонок: 10
Кількість числових колонок: 3
Кількість категоріальних колонок: 7
 - Бінарних: 3
 - Мультикатегоріальних: 4
Числові колонки: ['Age', 'Work_Experience', 'Family_Size']
Категоріальні колонки: ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1', 'Segmentation']
Бінарні колонки: ['Gender', 'Ever_Married', 'Graduated']
Мультикатегоріальні колонки: ['Profession', 'Spending_Score', 'Var_1', 'Segmentation']


In [189]:
from typing import Final

RANDOM_STATE: Final[int] = 42

train_df, test_df = train_test_split(raw_df, test_size=0.2, random_state=RANDOM_STATE, stratify=raw_df['Segmentation'])

# Define input columns and target column
input_cols = train_df.columns[1:-1]  # to drop 'ID' and 'Segmentation' columns
print('input_cols :', input_cols)

target_col: Final[str] = 'Segmentation'
print('target_col :', target_col)

# Prepare input data for the model
train_inputs = train_df[input_cols]
train_targets = train_df[target_col]

test_inputs = test_df[input_cols]
test_targets = test_df[target_col]

input_cols : Index(['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1'],
      dtype='object')
target_col : Segmentation


In [190]:
# Define numeric and categorical columns
numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
print(f'{numeric_cols=}')

binary_cols = ['Graduated', 'Ever_Married', 'Gender']
print(f'{binary_cols=}')

binary_cols_without_gender = ['Graduated', 'Ever_Married']
print(f'{binary_cols_without_gender=}')

gender_col = ['Gender']
print(f'{gender_col=}')

multi_categorical_cols = ['Profession', 'Spending_Score', 'Var_1']
print(f'{multi_categorical_cols=}')


numeric_cols=['Age', 'Work_Experience', 'Family_Size']
binary_cols=['Graduated', 'Ever_Married', 'Gender']
binary_cols_without_gender=['Graduated', 'Ever_Married']
gender_col=['Gender']
multi_categorical_cols=['Profession', 'Spending_Score', 'Var_1']


In [191]:
# Analyze missing values
print(raw_df.isnull().sum())

# Analyze categorical columns
for col in multi_categorical_cols:
    print(f'{col} : {raw_df[col].unique()}')

columns = raw_df.select_dtypes(include='object').columns

for column in columns:
    print(f'Column: {column}')
    print(raw_df[column].value_counts())
    print(raw_df[column].unique())
    print(f'Unique number: {raw_df[column].nunique()}', end='\n\n')

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64
Profession : ['Healthcare' 'Engineer' 'Lawyer' 'Entertainment' 'Artist' 'Executive'
 'Doctor' 'Homemaker' 'Marketing' nan]
Spending_Score : ['Low' 'Average' 'High']
Var_1 : ['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' nan 'Cat_5']
Column: Gender
Gender
Male      4417
Female    3651
Name: count, dtype: int64
['Male' 'Female']
Unique number: 2

Column: Ever_Married
Ever_Married
Yes    4643
No     3285
Name: count, dtype: int64
['No' 'Yes' nan]
Unique number: 2

Column: Graduated
Graduated
Yes    4968
No     3022
Name: count, dtype: int64
['No' 'Yes' nan]
Unique number: 2

Column: Profession
Profession
Artist           2516
Healthcare       1332
Entertainment     949
Engineer          699
Doctor            688
Lawyer       

In [192]:
# Preprocessing number columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

# Preprocessing binary columns
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder())  # same as df['Graduated'].map({'Yes': 1, 'No': 0})
])

# Preprocessing for column 'Gender' with OneHotEncoder
# We use OneHotEncoder to avoid assuming the order of values ('Male' and 'Female')
# As for simplicity we could leave it as is and use OrdinalEncoder in binary_transformer, but I want to make it more accurate
# since gender could affect the result I suppose.
gender_transformer = Pipeline(steps=[
    # We don't need imputer here, because there are no missing
    ('onehot', OneHotEncoder(sparse_output=False))
])

# Preprocessing categorical columns
multi_categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('bin', binary_transformer, binary_cols_without_gender),
        ('gender', gender_transformer, gender_col),
        ('cat', multi_categorical_transformer, multi_categorical_cols)
    ]
)

In [193]:
# Preprocess train data
X_train = preprocessor.fit_transform(train_inputs)
y_train = train_targets

# Preprocess test data
X_test = preprocessor.transform(test_inputs)
y_test = test_targets

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [194]:
# SMOTE works only on numeric data
# So lets transform it
X_numeric = numeric_transformer.fit_transform(train_inputs[numeric_cols])
y = train_targets

# Create SMOTE instance
smote = SMOTE(random_state=RANDOM_STATE)
X_train_smote, y_train_smote = smote.fit_resample(X_numeric, y)

X_test_smote = numeric_transformer.transform(test_inputs[numeric_cols])

original_counts = y.value_counts()
resampled_counts = pd.Series(y_train_smote).value_counts()

print("Було:")
for label, count in original_counts.items():
    print(f'Клас {label}: {count}')

print("\nСтало:")
for label, count in resampled_counts.items():
    print(f'Клас {label}: {count}')

Було:
Клас D: 1814
Клас A: 1578
Клас C: 1576
Клас B: 1486

Стало:
Клас A: 1814
Клас B: 1814
Клас C: 1814
Клас D: 1814


In [195]:
# SMOTENC works on numeric and categorical data (it is not designed to work with only categorical features.)
# So lets transform it :)

# Get indices of categorical features
cols = multi_categorical_cols + binary_cols
categorical_features_indices = [train_inputs.columns.get_loc(col) for col in cols]
print(cols)
print(categorical_features_indices)

# Create SMOTENC instance
smotenc = SMOTENC(categorical_features=categorical_features_indices, random_state=RANDOM_STATE)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)

original_counts = y.value_counts()
resampled_counts = pd.Series(y_train_smotenc).value_counts()

print("Було:")
for label, count in original_counts.items():
    print(f'Клас {label}: {count}')

print("\nСтало:")
for label, count in resampled_counts.items():
    print(f'Клас {label}: {count}')

['Profession', 'Spending_Score', 'Var_1', 'Graduated', 'Ever_Married', 'Gender']
[4, 6, 8, 3, 1, 0]
Було:
Клас D: 1814
Клас A: 1578
Клас C: 1576
Клас B: 1486

Стало:
Клас A: 1814
Клас B: 1814
Клас C: 1814
Клас D: 1814


In [196]:
# Model on Smote-Tomek data
smote_tomek = SMOTETomek(random_state=RANDOM_STATE)
X_train_smotetomek, y_train_smotetomek = smote_tomek.fit_resample(X_train, y_train)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [197]:
MAX_ITER = 1000

# Model on original data
log_reg = LogisticRegression(solver='liblinear', random_state=RANDOM_STATE, max_iter=MAX_ITER)
ovr_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', OneVsRestClassifier(log_reg))
])
ovr_model.fit(train_inputs, train_targets)
ovr_predictions = ovr_model.predict(test_inputs)

# Evaluation of the model on original data
print("Модель на оригінальних даних:")
print(classification_report(test_targets, ovr_predictions))

Модель на оригінальних даних:
              precision    recall  f1-score   support

           A       0.42      0.46      0.44       394
           B       0.42      0.17      0.24       372
           C       0.49      0.63      0.55       394
           D       0.65      0.76      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.48      1614
weighted avg       0.50      0.52      0.50      1614



In [198]:
#SMOTENC
pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTENC(categorical_features=categorical_features_indices, random_state=RANDOM_STATE)),
    ('classifier', OneVsRestClassifier(LogisticRegression(solver='liblinear', random_state=RANDOM_STATE, max_iter=MAX_ITER)))
])

pipeline.fit(train_inputs, train_targets)

ovr_predictions = pipeline.predict(test_inputs)
print("Модель на даних зі SMOTENC:")
print(classification_report(y_test, ovr_predictions))

Модель на даних зі SMOTENC:
              precision    recall  f1-score   support

           A       0.42      0.48      0.45       394
           B       0.41      0.25      0.31       372
           C       0.50      0.59      0.54       394
           D       0.68      0.72      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



In [199]:
# Model on SMOTE-Tomek data
pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote_tomek', SMOTETomek(random_state=RANDOM_STATE)),
    ('classifier', OneVsRestClassifier(LogisticRegression(solver='liblinear', random_state=RANDOM_STATE, max_iter=MAX_ITER)))
])

pipeline.fit(train_inputs, train_targets)

ovr_predictions = pipeline.predict(test_inputs)
print("Модель на даних зі SMOTE-Tomek:")
print(classification_report(y_test, ovr_predictions))

Модель на даних зі SMOTE-Tomek:
              precision    recall  f1-score   support

           A       0.42      0.47      0.44       394
           B       0.41      0.25      0.31       372
           C       0.50      0.59      0.54       394
           D       0.68      0.71      0.69       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



**Висновки:**

Клас D: 1814
Клас A: 1578
Клас C: 1576
Клас B: 1486


Для порівняння моделей я обрав метрику `f1-score`, оскільки вона враховує як точність, так і повноту моделі. Це дозволяє оцінити якість моделі більш точно, ніж просто за точністю або повнотою.

Модель на оригінальних даних показала найгірший результат серед трьох моделей. Проте це не показує, що вона набагато гірше, бо ми бачимо, що коренева різниця лише в предикшині B класу (що в цілому і очікувано, бо у нас в датасеті В класу менше, ніж інших). Тож використовуючи SMOTENC та SMOTE-Tomek ми отримали кращі результати, ніж на оригінальних даних. У нас дані не так, щоб і багато дисбалансовані, проте ми їх збалансували і бачим, що як раз на ось ту різницю +- воно і дало нам покращення.

Тепер чому така невелика різниця - я вважаю, що це може бути як раз із-за того, що не така і велика різниця в дизбалансі насправді. Ми бачимо, що класу D у нас найбільше (1814 інстансів), коли найменше - клабу B (1486 інстанстів). Але в моєму тут прикладі такий цікавий ефект, що якщо подивитись на інші метрики як recall & f1, то для класу D там на 0.01 менше значення у SMOTE-Tomek, ніж для SMOTENC. У моєму варіанті я же там окремо Gender опрацював як OneHotEncoder замість OrdinalEncoder, тому можливо це вплинуло на результати.